# DirectKinematics

Notebook to compute the position of the EE of the Kinova Gen3 lite from the joint positions.

In [ ]:
import math
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### Calcul de la cinématique directe. (direct kinematics)

The selected Python code defines a function named `dh` that calculates a Denavit-Hartenberg (DH) matrix. The DH matrix is a transformation matrix used in robotics to simplify the process of calculating the position and orientation of robot manipulators. 

The function `dh` takes four arguments: `theta`, `d`, `a`, and `alpha`. These parameters represent joint angle, link offset, link length, and link twist, respectively, in the DH convention. The parameters `d` and `a` are expected to be in millimeters, while `theta` and `alpha` are expected to be in degrees.

Inside the function, the `math.radians` function is used to convert `theta` and `alpha` from degrees to radians, as trigonometric functions in Python's `math` and `numpy` libraries expect angles to be in radians.

The function then returns a 4x4 numpy array, which represents the DH transformation matrix. This matrix is used to transform coordinates from one frame to another in a robotic manipulator. The matrix is composed of rotation and displacement components, which are calculated using the input parameters and basic trigonometric functions.

In [ ]:
# Fonction dh(theta, d, a, alpha)
# Arguments: d et a en mm; theta et alpha en degrées.
# Sortie: Matrice homogène.
# (en) Function dh(theta, d, a, alpha)
# Arguments: d and a in mm; theta and alpha in degrees.
# Output: Homogeneous matrix.
def dh(theta,d,a,alpha):
    theta = math.radians(theta)
    alpha = math.radians(alpha)
    return np.array([
        [np.cos(theta), -1*np.sin(theta)*np.cos(alpha),    np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta),    np.cos(theta)*np.cos(alpha), -1*np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [            0,                  np.sin(alpha),                  np.cos(alpha),               d],
        [            0,                              0,                              0,               1]
    ])

In [ ]:
# Fonction dk(x, y, z, w, p, r)
# Arguments: Theta 1, Theta 2 , Theta 3  en degrées.
# Sortie: Vecteur [x, y, z, w, p, r] en mm et en degrées.
# (en) Function dk(x, y, z, w, p, r)
# Arguments: Theta 1, Theta 2 , Theta 3 in degrees.
# Output: Vector [x, y, z, w, p, r] in mm and degrees.
def dk(t1,t2,t3):
    H10 = dh(t1,243.3,0,90)
    H21 = dh(t2+90,10,280,180)
    H32 = dh(t3+90,0,57,90)
    Htool3 = dh(0,480,0,0)

    H = np.matmul(H10, H21)
    H = np.matmul(H, H32)
    H = np.matmul(H, Htool3)

    if abs(H[2,0]) == 1:
        p = -H[2,0]*np.pi/2
        w = 0 # valeur arbitraire, on choisit w = 0
        r = np.degrees(np.arctan2(-H[2,0]*H[1,2], H[1,1])) #*180/np.pi
    else:
        p = np.arctan2(-H[2,0], math.sqrt(math.pow(H[0,0],2)+math.pow(H[1,0],2)))
        cp = np.cos(p)
        r = np.arctan2(H[1,0]/cp, H[0,0]/cp)
        w = np.arctan2(H[2,1]/cp, H[2,2]/cp)

        p = np.degrees(p)
        r = np.degrees(r)
        w = np.degrees(w)

    return np.array([H[0,3], H[1,3], H[2,3], w, p, r])

In [ ]:
# Essai de la fonction dk(t1,t2,t3)
# Test of the function dk(t1,t2,t3)
pose = dk(0,0,0)
print(pose)
# En entrant les valeurs suivantes dans votre fonction ik(x,y,z,w,p,r)
# vous devriez retrouver les angles passés en paramètre à dk(t1,t2,t3).
# Essayez avec plusieurs valeurs afin de valider vos calculs.
# (en) By entering the following values in your function ik(x,y,z,w,p,r)
# you should find the angles passed as parameter to dk(t1,t2,t3).
# Try with several values to validate your calculations.